# 6. Interrogation du DNS avec Python

### 1. On utilise le module Python socket afin de résoudre www.iutbeziers.fr en adresse IP.

In [1]:
import socket
print(socket.gethostbyname('www.iutbeziers.fr'))

194.199.227.80


### 2. On utilise le module dnspython afin de résoudre www.iutbeziers.fr en adresse IP et reciproquement.

In [63]:
import dns.resolver
import dns.reversename as rvrname

rslv = dns.resolver.Resolver()

ipqry = rslv.query('www.iutbeziers.fr')
ip  = qry[0].address
rvqry = rvrname.from_address(ip)
rv = rslv.query(rvqry,'PTR')

print("IP : ", ip )
print("URL: ", rv[0])

IP :  194.199.227.80
URL:  www.iutbeziers.fr.


### 3. On donne la liste des NS, des MX du domaine iutbeziers.fr.

#### NS.

In [61]:
import dns.resolver as rslv

dnsQry = rslv.query('iutbeziers.fr', 'NS')

for res in dnsQry:
    print(res)

scribe.iutbeziers.fr.


#### MX

In [62]:
import dns.resolver as rslv

dnsQry = rslv.query('iutbeziers.fr', 'MX')

for res in dnsQry:
    print(res)

10 mail.iutbeziers.fr.


### 4. On retrouve le resolver de votre container.

In [ ]:
import dns.resolver

rsvl = dns.resolver.Resolver()
    
print(rsvl.nameservers)

### 5. Utilisez le module dns2python afin de vérifier que les zones renvoyées en axfr par les DNS du domaine "zonetransfert.me." sont bien identiques/

# 7. Scapy un package Python pour tester votre sécurité

###  EXo1 SCAN de port: Script permettant de scanner les ports de 1 à 1024 des machines de la salle

In [1]:
from scapy.all import *
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

src_port = RandShort()

print("Scan en cours...")

for i in range(1, 20):
    dst_ip = "10.203.%d.1" % (i)

    
    for dst_port in range(22, 23):
        tcp_connect_scan_resp = sr1(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="S"),timeout=15, verbose=0 )

        if(str(type(tcp_connect_scan_resp))=="<class 'NoneType'>"):
            print("Port ", dst_port, "NoneType")
        elif(tcp_connect_scan_resp.haslayer(TCP)):
            if(tcp_connect_scan_resp.getlayer(TCP).flags == 0x12):
                send_rst = sr(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="AR"),timeout=10, verbose=0 )
                print("Host", dst_ip)
                print("Port ", dst_port, "isOpen")
            #elif (tcp_connect_scan_resp.getlayer(TCP).flags == 0x14):
             #   print( dst_ip, "P: ", dst_port, "isClosed")
                
    print("-----------------------------------")
           
print("Scan terminé.")


Scan en cours...
Host 10.203.1.1
Port  22 isOpen
-----------------------------------
Host 10.203.2.1
-----------------------------------
Host 10.203.3.1
-----------------------------------
Host 10.203.4.1
Port  22 isOpen
-----------------------------------
Host 10.203.5.1
Port  22 isOpen
-----------------------------------
Host 10.203.6.1
-----------------------------------
Host 10.203.7.1
-----------------------------------
Host 10.203.8.1
Port  22 isOpen
-----------------------------------
Host 10.203.9.1
Port  22 isOpen
-----------------------------------
Host 10.203.10.1
-----------------------------------
Host 10.203.11.1
Port  22 isOpen
-----------------------------------
Host 10.203.12.1
-----------------------------------
Host 10.203.13.1
-----------------------------------
Host 10.203.14.1
Port  22 isOpen
-----------------------------------
Host 10.203.15.1
Port  22 NoneType
-----------------------------------
Host 10.203.16.1
-----------------------------------
Host 10.203.17

### Exo2 Requête SNMP: Script permettant d'afficher et de recueillir desinformations sur les kernels Linux des serveurs de l'IUT ( Vérifiez que le port 161 udp est ouvert et passez votre requête )

In [4]:
from scapy.all import *
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

src_port = RandShort()
dst_port = 161
dst_ip = "10.255.255.135"

#p = IP(dst="10.255.255.135")/UDP(sport=161)/SNMP(community="publicbeziers",PDU=SNMPget(varbindlist=[SNMPvarbind(oid=ASN1_OID("1.3.6.1.2.1.1.1.0"))]))
#print(sr1(p))

udp_scan_resp = sr1(IP(dst=dst_ip)/UDP(dport=dst_port),timeout=10 )
    
if (str(type(udp_scan_resp))=="<class 'NoneType'>"):
    print("Port ", dst_port, "NoneType")
elif (udp_scan_resp.haslayer(UDP)):
    print("Port ", dst_port, "isOpen")
elif(udp_scan_resp.haslayer(ICMP)):
    if(int(udp_scan_resp.getlayer(ICMP).type)==3):
        print("Port ", dst_port, "Closed")
    elif(int(udp_scan_resp.getlayer(ICMP).type)==3 and int(udp_scan_resp.getlayer(ICMP).code) in [1,2,9,10,13]):
        print("Port ", dst_port, "Filtered")


Begin emission:
Finished sending 1 packets.

Received 7 packets, got 0 answers, remaining 1 packets
Port  161 NoneType


### Exo 3 Modifiez ce script qui sniffe le réseau avec une fonction de callback qui affiche les sources/dest IP afin qu'il affiche aussi les ports sources et destination ainsi que le type de paquets

In [14]:
from collections import Counter
from scapy.all import sniff
 
packet_counts = Counter()
 
#Packet #1: proto: TCP 192.168.16.2 ==> 10.255.255.135 sport: 45910 ==>  dport:80 
    
def custom_action(packet):
    key = tuple(sorted([packet[0][1].src, packet[0][1].dst]))
    packet_counts.update([key])
    packet.show()
    #return f"Packet #{sum(packet_counts.values())}: {packet[0][1].src} ==> {packet[0][1].dst}"
    return f"Packet #{sum(packet_counts.values())}: proto: {packet[0][1].proto} {packet[0][1].src} ==> {packet[0][1].dst} sport: {packet[0][1].sport}  ==> {packet[0][1].dport} "
    
sniff(filter="ip", prn=custom_action, count=10)
 
print("\n".join(f"{f'{key[0]} <--> {key[1]}'}: {count}" for key, count in packet_counts.items()))

###[ Ethernet ]### 
  dst       = 02:42:ac:12:00:02
  src       = 02:42:38:f0:f2:77
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 494
     id        = 65221
     flags     = DF
     frag      = 0
     ttl       = 63
     proto     = tcp
     chksum    = 0x7e64
     src       = 10.203.6.1
     dst       = 172.18.0.2
     \options   \
###[ TCP ]### 
        sport     = 40606
        dport     = 8888
        seq       = 890792378
        ack       = 1140458654
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 59
        chksum    = 0x8fae
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (6971879, 3584397720))]
###[ Raw ]### 
           load      = 'GET /static/base/images/favicon-busy-1.ico HTTP/1.1\r\nHost: 10.203.6.2:8888\r\nUser-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0\r\nAccept: text/html,application/xhtml+xml,app

###[ Ethernet ]### 
  dst       = 02:42:38:f0:f2:77
  src       = 02:42:ac:12:00:02
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 7141
     id        = 24157
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x3d6
     src       = 172.18.0.2
     dst       = 10.203.6.1
     \options   \
###[ TCP ]### 
        sport     = 8888
        dport     = 40554
        seq       = 903311515
        ack       = 3085382087
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 609
        chksum    = 0xd8b7
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3584401118, 6971932))]
###[ Raw ]### 
           load      = '\x81~\x1b\xad{"header": {"msg_id": "f131e477-ab650869017e6c0ceb9fda9b", "msg_type": "stream", "username": "root", "session": "e53220c9-a8422bd20e783dcc9e128eab", "date": "2019-04-08T15:51:20.894293Z", "vers

###[ Ethernet ]### 
  dst       = 02:42:38:f0:f2:77
  src       = 02:42:ac:12:00:02
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 7292
     id        = 24162
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x33a
     src       = 172.18.0.2
     dst       = 10.203.6.1
     \options   \
###[ TCP ]### 
        sport     = 8888
        dport     = 40554
        seq       = 903318604
        ack       = 3085382087
        dataofs   = 8
        reserved  = 0
        flags     = A
        window    = 609
        chksum    = 0xd94e
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3584401329, 6971985))]
###[ Raw ]### 
           load      = '\x81~0\xf1{"header": {"msg_id": "6a0c7f4f-d194f81813cd0cec05fcf276", "msg_type": "stream", "username": "root", "session": "e53220c9-a8422bd20e783dcc9e128eab", "date": "2019-04-08T15:51:21.103070Z", "version"